<a href="https://colab.research.google.com/github/mr-alamdari/NLP-Sequence-Models-Beginner/blob/main/NLP_Sequence_Models_Beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import re

In [34]:
!pip install -q -U trax

import trax

import trax.fastmath.numpy as np

from trax import layers as tl

In [35]:
a = np.array(325)

In [36]:
display(a)

DeviceArray(325, dtype=int32, weak_type=True)

In [37]:
type(a)

jaxlib.xla_extension.DeviceArray

In [38]:
def f(x):
  return 3 * x**2 - 32

In [39]:
d_a = f(a)
d_a

DeviceArray(316843, dtype=int32, weak_type=True)

In [40]:
grad_f = trax.fastmath.grad(fun=f)

In [41]:
type(grad_f)

function

In [49]:
def tweet_to_tensor(tweet, vocab_dict, unk_token='__UNK__', verbose=False):

    word_list = process_tweet(tweet)
    tensor_l = []
    unk_ID = vocab_dict[unk_token]

    for word in word_list:
        word_ID = vocab_dict[word] if word in vocab_dict else unk_ID
        tensor_l.append(word_ID) 
    
    return tensor_l

In [50]:
class Layer(object):
    def __init__(self):
        self.weights = None

    def init(self, input_signature, random_key):
        self.init_weights_and_state(input_signature, random_key)
        return self.weights

    def __call__(self, x):
        return self.forward(x)

In [ ]:
class Relu(Layer):
    def forward(self, x):
        return np.maximum(x,0)

In [51]:
class Dense(Layer):

    def __init__(self, n_units, init_stdev=0.1):
        
        self._n_units = n_units
        self._init_stdev = init_stdev

    def forward(self, x):
        dense = np.dot(x, self.weights) 
        return dense

    def init_weights_and_state(self, input_signature, random_key):
        input_shape = input_signature.shape
        w = self._init_stdev * trax.fastmath.random.normal(key = random_key, shape = (input_shape[-1], self._n_units))
        self.weights = w
        return self.weights

In [52]:
tmp_embed = np.array([[1,2,3], [4,5,6]])

display(np.mean(tmp_embed,axis=0))

display(np.mean(tmp_embed,axis=1))

DeviceArray([2.5, 3.5, 4.5], dtype=float32)

DeviceArray([2., 5.], dtype=float32)

In [55]:
def classifier(vocab, embedding_dim=256, output_dim=2, mode='train'):
    vocab_size=len(vocab)
    embed_layer = tl.Embedding(vocab_size=vocab_size, d_feature=embedding_dim)
    
    mean_layer = tl.Mean(axis=1)
    
    dense_output_layer = tl.Dense(n_units = output_dim)

    log_softmax_layer = tl.LogSoftmax()
    
    model = tl.Serial(
      embed_layer,
      mean_layer,
      dense_output_layer,
      log_softmax_layer
    )

    return model

In [58]:
from trax.supervised import training

def train_model(classifier, train_task, eval_task, n_steps, output_dir):
    training_loop = training.Loop(
                                classifier, # The learning model
                                train_task, # The training task
                                eval_task = eval_task, # The evaluation task
                                output_dir = output_dir) # The output directory

    training_loop.run(n_steps = n_steps)

    return training_loop

In [59]:
def compute_accuracy(preds, y, y_weights):

    is_pos =  preds[:, 1] > preds[:, 0] 
    is_pos_int = is_pos.astype(np.int32)
    correct = is_pos_int == y
    sum_weights = np.sum(y_weights)
    correct_float = correct.astype(np.float32)
    weighted_correct_float = correct_float * y_weights
    weighted_num_correct = np.sum(weighted_correct_float)
    accuracy = weighted_num_correct / sum_weights
    return accuracy, weighted_num_correct, sum_weights

In [60]:
def test_model(generator, model):
    accuracy = 0.
    total_num_correct = 0
    total_num_pred = 0
    for batch in generator: 
        inputs = batch[0]
        targets = batch[1]
        example_weight = batch[2]
        pred = model(inputs)
        batch_accuracy, batch_num_correct, batch_num_pred = compute_accuracy(pred, targets, example_weight)
        total_num_correct += batch_num_correct
        total_num_pred += batch_num_pred
    accuracy = total_num_correct / total_num_pred
    return accuracy

In [61]:
def predict(model, Vocab, sentence):
    inputs = np.array(tweet_to_tensor(sentence, vocab_dict=Vocab))
    
    inputs = inputs[None, :]  
    
    preds_probs = model(inputs)
    
    preds = int(preds_probs[0, 1] > preds_probs[0, 0])
    
    sentiment = "negative"
    if preds == 1:
        sentiment = 'positive'

    return preds, sentiment